In [349]:
import pandas as pd
import re
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import os

In [352]:
## Read CSVs
listofreadCSVbidtabs = []
for root,dirs, files in os.walk("ga_csv"):
    for file in files:
        filepath = os.path.join(root, file)
        listofreadCSVbidtabs.append(pd.read_csv(filepath))
print(len(listofreadCSVbidtabs))

683


In [356]:
print(dataframe.head())

         Proposal  Call Order  Section Number Section Description  Line  \
0  B1CBA2401581-0           2               1             ROADWAY     5   
1  B1CBA2401581-0           2               1             ROADWAY     5   
2  B1CBA2401581-0           2               1             ROADWAY     5   
3  B1CBA2401581-0           2               1             ROADWAY     5   
4  B1CBA2401581-0           2               1             ROADWAY     5   

       Item  Alternate Code           Item Description Quantity Unit  \
0  150-1000             NaN  TRAFFIC CONTROL - 0013813  1.00000   LS   
1  150-1000             NaN  TRAFFIC CONTROL - 0013813  1.00000   LS   
2  150-1000             NaN  TRAFFIC CONTROL - 0013813  1.00000   LS   
3  150-1000             NaN  TRAFFIC CONTROL - 0013813  1.00000   LS   
4  150-1000             NaN  TRAFFIC CONTROL - 0013813  1.00000   LS   

                                  Vendor Name       Bidder ID   Unit Price  \
0                E. R. SNELL CONTRACTO

In [355]:
dataframe = pd.read_csv("ga_csv/2024/B1CBA2401581-0.csv")
print(dataframe['Vendor Name'].unique())


['E. R. SNELL CONTRACTOR, INC.'
 'WRIGHT BROTHERS CONSTRUCTION COMPANY, INC.'
 'GEORGIA BRIDGE AND CONCRETE, LLC' 'UNITED INFRASTRUCTURE GROUP INC.'
 'C. W. MATTHEWS CONTRACTING CO., INC.']


In [317]:
# Clean Columns
def clean_numeric_columns(df, columns):
    def clean_string(s):
        if pd.isna(s):
            return s
        return re.sub(r'[^\d.]', '', str(s))

    for col in columns:
        if col in df.columns:
            df[col] = df[col].apply(clean_string)
            # Convert to float if possible
            df[col] = pd.to_numeric(df[col], errors='ignore')
        else:
            print(f"Warning: Column '{col}' not found in the DataFrame.")

    return df

In [318]:
def necessarycolumns(df):
    columnsforFeatureVector = ["Proposal","Item", "Quantity", "Vendor Name", "Unit Price", "Extension"]
    featurevectordataframe = df[columnsforFeatureVector]
    return featurevectordataframe

In [319]:
featurevectordataframe = necessarycolumns(sortedbidtab)
print(featurevectordataframe.head())

         Proposal      Item     Quantity                   Vendor Name  \
0  B1CBA2401581-0  150-1000      1.00000  E. R. SNELL CONTRACTOR, INC.   
1  B1CBA2401581-0  603-2024     15.00000  E. R. SNELL CONTRACTOR, INC.   
2  B1CBA2401581-0  603-2181    180.00000  E. R. SNELL CONTRACTOR, INC.   
3  B1CBA2401581-0  520-1173  1,310.00000  E. R. SNELL CONTRACTOR, INC.   
4  B1CBA2401581-0  603-7000    195.00000  E. R. SNELL CONTRACTOR, INC.   

    Unit Price    Extension  Bid Tab Rankings  
0  $425,000.00  $425,000.00                 0  
1       $99.75    $1,496.25                 0  
2       $53.75    $9,675.00                 0  
3      $140.00  $183,400.00                 0  
4        $5.45    $1,062.75                 0  


In [320]:
cleanedfeaturevectorDF = clean_numeric_columns(featurevectordataframe, ["Quantity", "Unit Price", "Extension"])
cleanedfeaturevectorDF.head()

/var/folders/3j/nswsh09j5rxcg8c7sj06k4080000gn/T/ipykernel_89193/3404092778.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(clean_string)
/var/folders/3j/nswsh09j5rxcg8c7sj06k4080000gn/T/ipykernel_89193/3404092778.py:12: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
/var/folders/3j/nswsh09j5rxcg8c7sj06k4080000gn/T/ipykernel_89193/3404092778.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

,Proposal,Item,Quantity,Vendor Name,Unit Price,Extension,Bid Tab Rankings
0,B1CBA2401581-0,150-1000,1.0,"E. R. SNELL CONTRACTOR, INC.",425000.00,425000.00,0
1,B1CBA2401581-0,603-2024,15.0,"E. R. SNELL CONTRACTOR, INC.",99.75,1496.25,0
2,B1CBA2401581-0,603-2181,180.0,"E. R. SNELL CONTRACTOR, INC.",53.75,9675.00,0
3,B1CBA2401581-0,520-1173,1310.0,"E. R. SNELL CONTRACTOR, INC.",140.00,183400.00,0
4,B1CBA2401581-0,603-7000,195.0,"E. R. SNELL CONTRACTOR, INC.",5.45,1062.75,0


In [321]:
individualcontractorDFs = dict()
for contractor in cleanedfeaturevectorDF['Vendor Name'].unique():
    individualcontractorDFs[contractor] = cleanedfeaturevectorDF[cleanedfeaturevectorDF["Vendor Name"] == contractor].reset_index(drop=True)
    individualcontractorDFs[contractor].dropna(subset=['Item'], inplace=True)
    df = individualcontractorDFs[contractor]
    duplicated_items = df[df['Item'].duplicated(keep=False)]["Item"].unique()
    #print(f"Duplicated Items: {duplicated_items}")
    for item in duplicated_items:
        subset = df[df["Item"] == item].reset_index(drop=True)
        #print(f"Subset: {subset}")
        itemdf = pd.DataFrame({
            "Proposal": [df["Proposal"].iloc[0]],
            "Item": [item],
            "Quantity": [subset["Quantity"].sum()],
            "Vendor Name": [df["Vendor Name"].iloc[0]],
            "Unit Price": [subset["Unit Price"].mean()], # NOTE: COME BACK TO THIS...THIS MAY BE AN ISSUE
            "Extension": [subset["Extension"].sum()]
        })
        #print(f"Item df: {itemdf}")
        df = df[df["Item"] != item]
        df = pd.concat([df, itemdf])
    individualcontractorDFs[contractor] = df   
    print(len(individualcontractorDFs[contractor]))
    

111
107
107
111
107


In [322]:
GADOT_payitem_index = list(pd.read_csv("PayItemIndex_2021.csv")["ITEM NO."])
print(len(GADOT_payitem_index))


4945


In [323]:
itemsEachContractorinProposal = dict()
for contractorname, contractor in individualcontractorDFs.items():
    GADOT_payitemsDF = pd.DataFrame({"Item": GADOT_payitem_index})
    allPayItems = pd.merge(GADOT_payitemsDF, contractor, on="Item", how="left")
    # Fill NaN values for specific columns
    allPayItems["Proposal"] = allPayItems["Proposal"].fillna(contractor["Proposal"].iloc[0])
    allPayItems["Vendor Name"] = allPayItems["Vendor Name"].fillna(contractor["Vendor Name"].iloc[0])
    allPayItems["Bid Tab Rankings"] = allPayItems["Bid Tab Rankings"].fillna(contractor["Bid Tab Rankings"].iloc[0])
    
    # Fill NaN values for numeric columns with 0
    numeric_columns = ["Quantity", "Unit Price", "Extension"]
    allPayItems[numeric_columns] = allPayItems[numeric_columns].fillna(0)
    itemsEachContractorinProposal[contractorname] = allPayItems

In [324]:
print(itemsEachContractorinProposal)

{'E. R. SNELL CONTRACTOR, INC.':               Item        Proposal  Quantity                   Vendor Name  \
0         001-1000  B1CBA2401581-0       0.0  E. R. SNELL CONTRACTOR, INC.   
1         001-1900  B1CBA2401581-0       0.0  E. R. SNELL CONTRACTOR, INC.   
2         001-1999  B1CBA2401581-0       0.0  E. R. SNELL CONTRACTOR, INC.   
3         001-2000  B1CBA2401581-0       0.0  E. R. SNELL CONTRACTOR, INC.   
4         001-2001  B1CBA2401581-0       0.0  E. R. SNELL CONTRACTOR, INC.   
...            ...             ...       ...                           ...   
4940  T-901-501-01  B1CBA2401581-0       0.0  E. R. SNELL CONTRACTOR, INC.   
4941  T-901-501-02  B1CBA2401581-0       0.0  E. R. SNELL CONTRACTOR, INC.   
4942  T-904-501-01  B1CBA2401581-0       0.0  E. R. SNELL CONTRACTOR, INC.   
4943  T-905-501-01  B1CBA2401581-0       0.0  E. R. SNELL CONTRACTOR, INC.   
4944  T-905-502-01  B1CBA2401581-0       0.0  E. R. SNELL CONTRACTOR, INC.   

      Unit Price  Extension  B

In [325]:
print(itemsEachContractorinProposal['E. R. SNELL CONTRACTOR, INC.'].columns)

Index(['Item', 'Proposal', 'Quantity', 'Vendor Name', 'Unit Price',
       'Extension', 'Bid Tab Rankings'],
      dtype='object')


In [326]:
# Need to flatten each dataframe of each contractor to form feature vector
# Combine vectors to form single example and single label
# Then iterator across all BidX contracts
# Split into Training, Validation, and Testing from there

def getContractor(dataframe: pd.DataFrame):
    return dataframe.iat[0, 3]
def getProposalID(dataframe: pd.DataFrame):
    return dataframe.iat[0, 1]
def getRanking(dataframe: pd.DataFrame):
    return int(dataframe.iat[0, 6])
print(getContractor(itemsEachContractorinProposal['E. R. SNELL CONTRACTOR, INC.']))
print(getProposalID(itemsEachContractorinProposal['E. R. SNELL CONTRACTOR, INC.']))
print(getRanking(itemsEachContractorinProposal['GEORGIA BRIDGE AND CONCRETE, LLC']))

E. R. SNELL CONTRACTOR, INC.
B1CBA2401581-0
2


In [327]:
def scaleNumericColumns(contractor_name: str):
    numeric_features = ['Quantity', 'Unit Price', 'Extension']
    categorical_features = ['Item', 'Vendor Name', 'Bid Tab Rankings']
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
        ])
    pipeline = Pipeline([('preprocessor', preprocessor)])
    
    feature_vector = pipeline.fit_transform(itemsEachContractorinProposal[contractor_name])
    return pd.DataFrame(feature_vector)

In [328]:
proposalScaledNumeric = pd.DataFrame()
finalbids = dict()
for contractorname, dataframe in itemsEachContractorinProposal.items():
    finalbids[contractorname] = dataframe['Extension'].sum()
    feature_df = scaleNumericColumns(contractorname)
    scaledNumericDF = dataframe
    scaledNumericDF["Quantity"] = feature_df[0]
    scaledNumericDF["Unit Price"] = feature_df[1]
    scaledNumericDF["Extension"] = feature_df[2]
    print(scaledNumericDF.head())
    proposalScaledNumeric = pd.concat([proposalScaledNumeric, scaledNumericDF], axis=0)


       Item        Proposal  Quantity                   Vendor Name  \
0  001-1000  B1CBA2401581-0 -0.035475  E. R. SNELL CONTRACTOR, INC.   
1  001-1900  B1CBA2401581-0 -0.035475  E. R. SNELL CONTRACTOR, INC.   
2  001-1999  B1CBA2401581-0 -0.035475  E. R. SNELL CONTRACTOR, INC.   
3  001-2000  B1CBA2401581-0 -0.035475  E. R. SNELL CONTRACTOR, INC.   
4  001-2001  B1CBA2401581-0 -0.035475  E. R. SNELL CONTRACTOR, INC.   

   Unit Price  Extension  Bid Tab Rankings  
0   -0.022908  -0.048726               0.0  
1   -0.022908  -0.048726               0.0  
2   -0.022908  -0.048726               0.0  
3   -0.022908  -0.048726               0.0  
4   -0.022908  -0.048726               0.0  
       Item        Proposal  Quantity  \
0  001-1000  B1CBA2401581-0 -0.036096   
1  001-1900  B1CBA2401581-0 -0.036096   
2  001-1999  B1CBA2401581-0 -0.036096   
3  001-2000  B1CBA2401581-0 -0.036096   
4  001-2001  B1CBA2401581-0 -0.036096   

                                  Vendor Name  Unit Pric

In [329]:
proposalScaledNumeric.shape

(24725, 7)

In [330]:
print(proposalScaledNumeric.columns)

Index(['Item', 'Proposal', 'Quantity', 'Vendor Name', 'Unit Price',
       'Extension', 'Bid Tab Rankings'],
      dtype='object')


In [331]:
def encodeCategoricalColumns(dataframe):
    numeric_features = ['Quantity', 'Unit Price', 'Extension']
    categorical_features = ['Item', 'Vendor Name', 'Bid Tab Rankings']
    item_encoder = LabelEncoder()
    dataframe['Item'] = item_encoder.fit_transform(dataframe['Item'])
    vendor_encoder = OneHotEncoder(sparse_output=False)
    preprocessor = ColumnTransformer(
        transformers=[
            ('vendor_encoder', vendor_encoder, ['Vendor Name']),
            ('num', 'passthrough', ['Item','Proposal', 'Quantity', 'Unit Price', 'Extension'])
        ])
    pipeline = Pipeline([('preprocessor', preprocessor)])
    encoded_data = pipeline.fit_transform(dataframe)

    fitted_vendor_encoder = pipeline.named_steps['preprocessor'].named_transformers_['vendor_encoder']
    vendor_feature_names = fitted_vendor_encoder.get_feature_names_out(['Vendor Name'])

    all_feature_names = list(vendor_feature_names) + ['Item','Proposal', 'Quantity', 'Unit Price', 'Extension']

    
    return pd.DataFrame(encoded_data, columns = all_feature_names)

In [332]:
encodedDataFrame = encodeCategoricalColumns(proposalScaledNumeric)

In [333]:
print(encodedDataFrame.head())

  Vendor Name_C. W. MATTHEWS CONTRACTING CO., INC.  \
0                                              0.0   
1                                              0.0   
2                                              0.0   
3                                              0.0   
4                                              0.0   

  Vendor Name_E. R. SNELL CONTRACTOR, INC.  \
0                                      1.0   
1                                      1.0   
2                                      1.0   
3                                      1.0   
4                                      1.0   

  Vendor Name_GEORGIA BRIDGE AND CONCRETE, LLC  \
0                                          0.0   
1                                          0.0   
2                                          0.0   
3                                          0.0   
4                                          0.0   

  Vendor Name_UNITED INFRASTRUCTURE GROUP INC.  \
0                                          0.0 

In [334]:
print(finalbids)
dollarlabels = list(finalbids.values())

{'E. R. SNELL CONTRACTOR, INC.': 16444620.76, 'WRIGHT BROTHERS CONSTRUCTION COMPANY, INC.': 17529804.1, 'GEORGIA BRIDGE AND CONCRETE, LLC': 18631145.18, 'UNITED INFRASTRUCTURE GROUP INC.': 18923907.04, 'C. W. MATTHEWS CONTRACTING CO., INC.': 20214514.189999998}


In [335]:
print(dollarlabels)

[16444620.76, 17529804.1, 18631145.18, 18923907.04, 20214514.189999998]


In [336]:
# Sort the dictionary items based on values (bid amounts)
sorted_bids = sorted(finalbids.items(), key=lambda x: x[1])

# Create a new dictionary with places
placed_bids = {contractor: {'amount': amount, 'place': place + 1} 
               for place, (contractor, amount) in enumerate(sorted_bids)}

# Print the result
for contractor, info in placed_bids.items():
    print(f"{contractor}: Place {info['place']}, Bid Amount: ${info['amount']:,.2f}")

E. R. SNELL CONTRACTOR, INC.: Place 1, Bid Amount: $16,444,620.76
WRIGHT BROTHERS CONSTRUCTION COMPANY, INC.: Place 2, Bid Amount: $17,529,804.10
GEORGIA BRIDGE AND CONCRETE, LLC: Place 3, Bid Amount: $18,631,145.18
UNITED INFRASTRUCTURE GROUP INC.: Place 4, Bid Amount: $18,923,907.04
C. W. MATTHEWS CONTRACTING CO., INC.: Place 5, Bid Amount: $20,214,514.19


In [337]:
contractors = dict()
for column in encodedDataFrame.columns:
    if "Vendor Name" in column:
        contractors[column[12:]] = encodedDataFrame[encodedDataFrame[column] > 0].loc[:,["Quantity", "Unit Price"]]
print(contractors)

{'C. W. MATTHEWS CONTRACTING CO., INC.':        Quantity Unit Price
19780 -0.035942  -0.021915
19781 -0.035942  -0.021915
19782 -0.035942  -0.021915
19783 -0.035942  -0.021915
19784 -0.035942  -0.021915
...         ...        ...
24720 -0.035942  -0.021915
24721 -0.035942  -0.021915
24722 -0.035942  -0.021915
24723 -0.035942  -0.021915
24724 -0.035942  -0.021915

[4945 rows x 2 columns], 'E. R. SNELL CONTRACTOR, INC.':       Quantity Unit Price
0    -0.035475  -0.022908
1    -0.035475  -0.022908
2    -0.035475  -0.022908
3    -0.035475  -0.022908
4    -0.035475  -0.022908
...        ...        ...
4940 -0.035475  -0.022908
4941 -0.035475  -0.022908
4942 -0.035475  -0.022908
4943 -0.035475  -0.022908
4944 -0.035475  -0.022908

[4945 rows x 2 columns], 'GEORGIA BRIDGE AND CONCRETE, LLC':        Quantity Unit Price
9890  -0.035942  -0.022834
9891  -0.035942  -0.022834
9892  -0.035942  -0.022834
9893  -0.035942  -0.022834
9894  -0.035942  -0.022834
...         ...        ...
14830 -0.03594

In [339]:
winningbidamount = min(dollarlabels)

In [343]:
def createVectorLabelPairs(proposals: dict):
    proposal = encodedDataFrame['Proposal'].iloc[0]
    proposals[proposal] = []
    for contractorname, contractordataframe in contractors.items():
        proposals[proposal].append((contractorname, contractordataframe.to_numpy().flatten(), (placed_bids[contractorname]['place'], winningbidamount)))
    return proposals


In [344]:
proposals = dict()
proposals = createVectorLabelPairs(proposals)

In [345]:
print(proposals)

{'B1CBA2401581-0': [('C. W. MATTHEWS CONTRACTING CO., INC.', array([-0.03594155529929083, -0.02191542404445782, -0.03594155529929083,
       ..., -0.02191542404445782, -0.03594155529929083,
       -0.02191542404445782], dtype=object), (5, 16444620.76)), ('E. R. SNELL CONTRACTOR, INC.', array([-0.035475424571911766, -0.022907726859653955,
       -0.035475424571911766, ..., -0.022907726859653955,
       -0.035475424571911766, -0.022907726859653955], dtype=object), (1, 16444620.76)), ('GEORGIA BRIDGE AND CONCRETE, LLC', array([-0.03594155529929083, -0.022833777324449255, -0.03594155529929083,
       ..., -0.022833777324449255, -0.03594155529929083,
       -0.022833777324449255], dtype=object), (3, 16444620.76)), ('UNITED INFRASTRUCTURE GROUP INC.', array([-0.035475424571911766, -0.0296931997251182, -0.035475424571911766,
       ..., -0.0296931997251182, -0.035475424571911766,
       -0.0296931997251182], dtype=object), (4, 16444620.76)), ('WRIGHT BROTHERS CONSTRUCTION COMPANY, INC.', arra

In [ ]:
# Training -> quantity and unit price -> have model predict place and winnning bid amount 

In [357]:
def createXtrainYtrain(proposals):
    X_data = []
    y_data = []
    for proposal, infolist in proposals.items():
        for infotuple in infolist:
            X_data.append(infotuple[1])
            y_data.append(infotuple[2])
    return np.array(X_data), np.array(y_data)

In [358]:
print(createXtrainYtrain(proposals))

(array([[-0.03594155529929083, -0.02191542404445782, -0.03594155529929083,
        ..., -0.02191542404445782, -0.03594155529929083,
        -0.02191542404445782],
       [-0.035475424571911766, -0.022907726859653955,
        -0.035475424571911766, ..., -0.022907726859653955,
        -0.035475424571911766, -0.022907726859653955],
       [-0.03594155529929083, -0.022833777324449255,
        -0.03594155529929083, ..., -0.022833777324449255,
        -0.03594155529929083, -0.022833777324449255],
       [-0.035475424571911766, -0.0296931997251182,
        -0.035475424571911766, ..., -0.0296931997251182,
        -0.035475424571911766, -0.0296931997251182],
       [-0.036096031114238716, -0.023933794089566248,
        -0.036096031114238716, ..., -0.023933794089566248,
        -0.036096031114238716, -0.023933794089566248]], dtype=object), array([[5.00000000e+00, 1.64446208e+07],
       [1.00000000e+00, 1.64446208e+07],
       [3.00000000e+00, 1.64446208e+07],
       [4.00000000e+00, 1.64446208e